In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

### Loading the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct"
)

### Loading the model

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map='cuda',
    torch_dtype='auto',
    trust_remote_code=True,
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.88s/it]
c:\Users\rafae\Documents\Projects\hands-on-llms-rafael\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rafae\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develo

### Creating the pipeline

In [14]:
generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Testing our pipeline

In [ ]:
prompt="Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt, use_cache=False)

print(output[0]['generated_text']) # type: ignore

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Mention the steps you're taking to prevent it in the future.

Email:

Subject: Sincere Apologies for the Gardening Mishap

Dear Sarah,

I hope this email finds you well
